## Notebook purpose

1. Reading stock data from various available free API services.

2. Analyse the quality of data 


## Importing libs


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os 
import sys
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from datetime import datetime
import mplfinance as mfin
import matplotlib.pyplot as plt
import cufflinks as cf
from plotly.offline import iplot, init_notebook_mode
import altair as alt
import shelve
from functools import partial
import plotly.graph_objects as go
s = shelve.open('data_shelf.db')

In [3]:
sys.path.append('../.')

In [4]:
from data import *
import data

## Downloading data 

In [5]:
symbols = ['AAPL', 'TSLA', "GOOG", 'FB', 'GS']
countries = ['United States', 'India', 'China']
supported_frameworks = datadownloader.supported_frameworks

In [6]:
def download_and_display(symbol = widgets.Dropdown(options=symbols,
                                                    value='AAPL',
                                                    description='Stock Ticker:',
                                                    disabled=False,
                                                   ), 
                         start_date = widgets.DatePicker(description='Pick start date', 
                                                         value=datetime.strptime('01/01/2021', '%d/%m/%Y'),
                                                         disabled=False),
                         end_date = widgets.DatePicker(description='Pick end date', 
                                                       value=datetime.strptime('01/10/2021', '%d/%m/%Y'),
                                                       disabled=False),
                         country = widgets.Dropdown(options=countries,
                                                    value='United States',
                                                    description='Country:',
                                                    disabled=False,
                                                   ),
                         framework = widgets.Dropdown(options=supported_frameworks,
                                                    value='yahoo_fin',
                                                    description='API Framework:',
                                                    disabled=False,
                                                   ),
                        ):
    ticker = symbol
    start_date = start_date.strftime('%d/%m/%Y')
    end_date = end_date.strftime('%d/%m/%Y')
    df = datadownloader.get_daily_historical_data(ticker, 
                                                  start_date=start_date, 
                                                  end_date=end_date, 
                                                  country=country, 
                                                  framework=framework)
    print(start_date)
    df['ticker'] = ticker
    print(df.head())
    s['df'] = df

In [10]:
downloader = interactive(download_and_display)

In [11]:
display(downloader)

interactive(children=(Dropdown(description='Stock Ticker:', options=('AAPL', 'TSLA', 'GOOG', 'FB', 'GS'), valu…

In [31]:
def visualize_chart(start_date = widgets.DatePicker(description='Pick start date', 
                                               value=datetime.strptime('01/01/2021', '%d/%m/%Y'),
                                               disabled=False),
                   end_date = widgets.DatePicker(description='Pick start date', 
                                               value=datetime.strptime('01/10/2021', '%d/%m/%Y'),
                                               disabled=False)):
    
    df = s['df'].loc[start_date.strftime('%d/%m/%Y'):].reset_index().rename(columns={'index':'date'})
    fig = go.Figure(data=[go.Candlestick(x=df['date'],
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'])])
    fig.update_layout(xaxis_rangeslider_visible=False)
    fig.update_layout(
    title=f"Stock Chart of {df.ticker.iloc[0]}",
    xaxis_title="date",
    yaxis_title="Price")
    fig.show()
#     qf = cf.QuantFig(df, title="Apple's stock price in 2021", name='AAPL')
#     qf.iplot()
    


In [32]:
visualizer = interactive(visualize_chart)

In [33]:
display(visualizer)

interactive(children=(DatePicker(value=datetime.datetime(2021, 1, 1, 0, 0), description='Pick start date'), Da…